# Step 10 — Audit Trail Module

**What we built**: An `AuditModule` that logs structured audit metadata per `invoke()` call — PII-safe by default.

**Why it matters**: Federal compliance (NIST 800-53 AU-3) requires an audit trail for every LLM interaction. Thousands of autonomous agents need observable interactions without creating new PII exposure. The audit module captures *what happened* (provider, model, message count, tool calls) without capturing *what was said* (message content, response content) — unless explicitly opted in.

**Key decisions**:
- **D-070**: Structured logging (same pattern as telemetry — parseable by log aggregation systems)
- **D-072**: Fields: provider, model, message_count, stop_reason, tools_provided (conditional), tool_calls (conditional), content_length
- **D-073**: No raw content by default — PII safety for federal environments
- **D-074**: `include_messages` and `include_response` — explicit opt-in, logged at DEBUG level (double opt-in)

**Stack position**: `Telemetry → Audit → Retry → Fallback → RateLimit → Adapter`

**New shared infrastructure**: `_logging.py` — `log_structured()`, `validate_log_level()`, `_sanitize()`

In [ ]:
# Setup: ensure arcllm is importable
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..', 'src')))

---
## 1. What Gets Logged (Default — PII-Safe)

Every `invoke()` call produces one structured audit log line:

```
INFO  Audit | provider=anthropic model=claude-sonnet-4-20250514 message_count=3
              stop_reason=end_turn content_length=142
```

**Always present**: provider, model, message_count, stop_reason, content_length

**Conditional**: `tools_provided` (only when tools arg is not None), `tool_calls` (only when response has tool_calls)

**Never logged by default**: Raw message content, raw response content (PII safety)

### Audit vs. Telemetry — Different Concerns

| | Telemetry | Audit |
|---|-----------|-------|
| **Purpose** | Performance + cost | Compliance + debugging |
| **Key fields** | duration_ms, tokens, cost_usd | message_count, tools, content_length |
| **Overlap** | provider, model, stop_reason | provider, model, stop_reason |
| **PII risk** | None (only numbers) | Protected (content opt-in) |

---
## 2. Constructing AuditModule

In [ ]:
from unittest.mock import AsyncMock, MagicMock
from arcllm.modules.audit import AuditModule
from arcllm.types import LLMProvider, LLMResponse, Message, Tool, ToolCall, Usage

# Create a mock inner adapter
inner = MagicMock(spec=LLMProvider)
inner.name = "anthropic"
inner.model_name = "claude-sonnet-4-20250514"
inner.invoke = AsyncMock(return_value=LLMResponse(
    content="Hello there!",
    usage=Usage(input_tokens=100, output_tokens=50, total_tokens=150),
    model="claude-sonnet-4-20250514",
    stop_reason="end_turn",
))

# Default config — PII-safe
module = AuditModule({}, inner)

print(f"Module type:        {type(module).__name__}")
print(f"Provider (inner):   {module.name}")
print(f"Model (inner):      {module.model_name}")
print(f"Include messages:   {module._include_messages}")
print(f"Include response:   {module._include_response}")
print(f"Log level:          {module._log_level} (20=INFO)")

---
## 3. Basic Audit Logging

In [ ]:
import logging

# Set up logging to see output
handler = logging.StreamHandler()
handler.setFormatter(logging.Formatter("%(levelname)s %(name)s: %(message)s"))
audit_logger = logging.getLogger("arcllm.modules.audit")
audit_logger.addHandler(handler)
audit_logger.setLevel(logging.INFO)

messages = [
    Message(role="system", content="You are a helpful assistant."),
    Message(role="user", content="What is 2+2?"),
]

result = await module.invoke(messages)

audit_logger.removeHandler(handler)

print(f"\nResponse: {result.content}")
print(f"\nNotice: NO message content or response content in the audit log above")

The log shows:
- `provider=anthropic` — which provider
- `model=claude-sonnet-4-20250514` — which model
- `message_count=2` — how many messages sent
- `stop_reason=end_turn` — why the model stopped
- `content_length=12` — response length in chars (`len("Hello there!")` = 12)

**Not logged**: "You are a helpful assistant", "What is 2+2?", "Hello there!" (PII-safe)

---
## 4. Conditional Fields: Tools and Tool Calls

Tool-related fields appear **only when relevant** — reducing log noise.

In [ ]:
# Response WITH tool calls
tool_response = LLMResponse(
    content=None,
    tool_calls=[
        ToolCall(id="call_1", name="search", arguments={"query": "test"}),
        ToolCall(id="call_2", name="calc", arguments={"expr": "1+1"}),
    ],
    usage=Usage(input_tokens=20, output_tokens=10, total_tokens=30),
    model="claude-sonnet-4-20250514",
    stop_reason="tool_use",
)

# Tools provided to the model
tools = [
    Tool(name="search", description="Search the web", parameters={"type": "object"}),
]

inner_tools = MagicMock(spec=LLMProvider)
inner_tools.name = "anthropic"
inner_tools.invoke = AsyncMock(return_value=tool_response)
mod_tools = AuditModule({}, inner_tools)

audit_logger.addHandler(handler)
audit_logger.setLevel(logging.INFO)

print("=== With tools and tool_calls ===")
await mod_tools.invoke(messages, tools=tools)

audit_logger.removeHandler(handler)

In [ ]:
# Without tools — fields omitted entirely
inner_no_tools = MagicMock(spec=LLMProvider)
inner_no_tools.name = "anthropic"
inner_no_tools.invoke = AsyncMock(return_value=LLMResponse(
    content="Just text.",
    usage=Usage(input_tokens=10, output_tokens=5, total_tokens=15),
    model="claude-sonnet-4-20250514",
    stop_reason="end_turn",
))
mod_no_tools = AuditModule({}, inner_no_tools)

audit_logger.addHandler(handler)
audit_logger.setLevel(logging.INFO)

print("=== Without tools (fields omitted) ===")
await mod_no_tools.invoke(messages)

audit_logger.removeHandler(handler)
print("\nNotice: 'tools_provided' and 'tool_calls' are absent")

The conditional logic uses `None` omission in `log_structured()`:
```python
tools_provided=len(tools) if tools is not None else None,  # None → omitted
tool_calls=len(response.tool_calls) if response.tool_calls else None,  # [] or None → omitted
```

---
## 5. PII Safety — The Default

The core compliance requirement: audit trail exists, but doesn't create new PII exposure.

In [ ]:
import io

# Capture ALL log output (even DEBUG)
stream = io.StringIO()
debug_handler = logging.StreamHandler(stream)
debug_handler.setLevel(logging.DEBUG)
audit_logger.addHandler(debug_handler)
audit_logger.setLevel(logging.DEBUG)

# Messages with sensitive content
sensitive_messages = [
    Message(role="system", content="You are a medical assistant."),
    Message(role="user", content="Patient John Smith, SSN 123-45-6789, has diabetes."),
]

sensitive_response = LLMResponse(
    content="Based on the patient's condition, I recommend...",
    usage=Usage(input_tokens=50, output_tokens=30, total_tokens=80),
    model="claude-sonnet-4-20250514",
    stop_reason="end_turn",
)

inner_pii = MagicMock(spec=LLMProvider)
inner_pii.name = "anthropic"
inner_pii.invoke = AsyncMock(return_value=sensitive_response)

# Default module — PII safe
mod_safe = AuditModule({}, inner_pii)
await mod_safe.invoke(sensitive_messages)

log_output = stream.getvalue()
audit_logger.removeHandler(debug_handler)

# Verify PII is NOT in the log
print("=== PII Safety Check ===")
print(f"Log output: {log_output.strip()}")
print()
print(f"Contains 'John Smith':     {'John Smith' in log_output}")
print(f"Contains 'SSN':            {'SSN' in log_output}")
print(f"Contains '123-45-6789':    {'123-45-6789' in log_output}")
print(f"Contains 'diabetes':       {'diabetes' in log_output}")
print(f"Contains 'recommend':      {'recommend' in log_output}")
print(f"Contains 'message_count':  {'message_count' in log_output}")
print(f"Contains 'content_length': {'content_length' in log_output}")

The audit log tells you *that* 2 messages were sent and the response was ~48 chars, without revealing *what* was in them.

---
## 6. Opt-In Content Logging (Double Opt-In)

For dev/staging environments where logging raw content is acceptable, content logging requires **two conditions**:
1. Config flag (`include_messages=True` / `include_response=True`)
2. Logger at DEBUG level

This "double opt-in" prevents accidental PII exposure.

In [ ]:
# Content logging with include_messages + include_response
inner_content = MagicMock(spec=LLMProvider)
inner_content.name = "anthropic"
inner_content.invoke = AsyncMock(return_value=LLMResponse(
    content="Hello there!",
    usage=Usage(input_tokens=10, output_tokens=5, total_tokens=15),
    model="claude-sonnet-4-20250514",
    stop_reason="end_turn",
))

mod_content = AuditModule(
    {"include_messages": True, "include_response": True},
    inner_content,
)

# Test 1: At INFO level — content NOT logged (second condition fails)
stream1 = io.StringIO()
h1 = logging.StreamHandler(stream1)
h1.setLevel(logging.INFO)
audit_logger.addHandler(h1)
audit_logger.setLevel(logging.INFO)

print("=== At INFO (content hidden despite include_* = True) ===")
await mod_content.invoke(messages)
log1 = stream1.getvalue()
print(f"Contains message content: {'You are a helpful' in log1}")
print(f"Contains response content: {'Hello there' in log1}")
audit_logger.removeHandler(h1)

# Test 2: At DEBUG level — content IS logged (both conditions met)
stream2 = io.StringIO()
h2 = logging.StreamHandler(stream2)
h2.setLevel(logging.DEBUG)
audit_logger.addHandler(h2)
audit_logger.setLevel(logging.DEBUG)

print("\n=== At DEBUG (content visible) ===")
await mod_content.invoke(messages)
log2 = stream2.getvalue()
print(f"Contains message content: {'helpful' in log2}")
print(f"Contains response content: {'Hello there' in log2}")
audit_logger.removeHandler(h2)
audit_logger.setLevel(logging.WARNING)  # Reset

The implementation:
```python
if self._include_messages and logger.isEnabledFor(logging.DEBUG):
    logger.debug("Audit messages | %s", _sanitize(str(messages)))
if self._include_response and logger.isEnabledFor(logging.DEBUG):
    logger.debug("Audit response | %s", _sanitize(str(response.content)))
```

The `isEnabledFor()` guard prevents expensive string building when DEBUG is disabled.

---
## 7. The Shared `_logging.py` Helper

A DRY extraction created during code review — shared by both TelemetryModule and AuditModule.

In [ ]:
from arcllm.modules._logging import log_structured, validate_log_level, _sanitize

print("=== _sanitize() — prevents log injection ===")
print(f"  Clean:     {repr(_sanitize('normal-model'))}")
print(f"  Newline:   {repr(_sanitize('evil\nINJECTED'))}")
print(f"  CR:        {repr(_sanitize('evil\rINJECTED'))}")
print(f"  Tab:       {repr(_sanitize('evil\tINJECTED'))}")
print(f"  Non-str:   {repr(_sanitize(42))}")

In [ ]:
print("=== validate_log_level() — shared validation ===")
print(f"  Default:    {validate_log_level({})} (INFO=20)")
print(f"  Explicit:   {validate_log_level({'log_level': 'DEBUG'})} (DEBUG=10)")
print(f"  Custom default: {validate_log_level({}, default='WARNING')} (WARNING=30)")

from arcllm.exceptions import ArcLLMConfigError
try:
    validate_log_level({"log_level": "NOPE"})
except ArcLLMConfigError as e:
    print(f"  Invalid:    {e}")

In [ ]:
# log_structured() — the core helper
test_logger = logging.getLogger("demo.structured")
h = logging.StreamHandler()
h.setFormatter(logging.Formatter("%(levelname)s: %(message)s"))
test_logger.addHandler(h)
test_logger.setLevel(logging.INFO)

print("=== log_structured() features ===")
print()

# None values omitted
print("1. None values omitted:")
log_structured(test_logger, logging.INFO, "Test",
    present="yes", missing=None, count=42)

# Floats formatted to 6 decimals
print("\n2. Float formatting (6 decimals):")
log_structured(test_logger, logging.INFO, "Test",
    cost_usd=0.00105)

# Strings sanitized
print("\n3. String sanitization:")
log_structured(test_logger, logging.INFO, "Test",
    model="evil\nINJECTED")

# Level gating (no output)
print("\n4. Level gating (DEBUG at INFO level → silent):")
log_structured(test_logger, logging.DEBUG, "Test",
    should_not="appear")
print("   (nothing above — level too low)")

test_logger.removeHandler(h)

---
## 8. Config Validation

AuditModule validates config keys **strictly** — catches typos at construction time.

In [ ]:
from arcllm.modules.audit import _VALID_CONFIG_KEYS

inner = MagicMock(spec=LLMProvider)
inner.name = "test"

print(f"Valid config keys: {sorted(_VALID_CONFIG_KEYS - {'enabled'})}")
print()

# Typo caught at construction
try:
    AuditModule({"include_mesages": True}, inner)  # Missing 's'
except ArcLLMConfigError as e:
    print(f"Typo caught: {e}")

# Invalid log level
try:
    AuditModule({"log_level": "NOPE"}, inner)
except ArcLLMConfigError as e:
    print(f"\nBad level:  {e}")

The `_VALID_CONFIG_KEYS` pattern was applied after code review — prevents silent misconfiguration where a typo'd key is silently ignored.

---
## 9. Content Length Calculation

In [ ]:
# Text response → len(content)
text_resp = LLMResponse(
    content="Hello there!",  # 12 chars
    usage=Usage(input_tokens=10, output_tokens=5, total_tokens=15),
    model="test", stop_reason="end_turn",
)

# Tool call response → content is None → length 0
tool_resp = LLMResponse(
    content=None,
    tool_calls=[ToolCall(id="1", name="search", arguments={"q": "test"})],
    usage=Usage(input_tokens=10, output_tokens=5, total_tokens=15),
    model="test", stop_reason="tool_use",
)

print(f"Text response: content_length = {len(text_resp.content or '')}")
print(f"Tool response: content_length = {len(tool_resp.content or '')}")
print()
print("Formula: len(response.content) if response.content else 0")

---
## 10. Log Injection Prevention

The `_sanitize()` function escapes control characters that could corrupt structured logs.

In [ ]:
# Model name with injection attempt
injected_response = LLMResponse(
    content="ok",
    usage=Usage(input_tokens=10, output_tokens=5, total_tokens=15),
    model="gpt-4\nINJECTED LOG LINE",  # Newline injection
    stop_reason="end_turn",
)

inner_inject = MagicMock(spec=LLMProvider)
inner_inject.name = "test"
inner_inject.invoke = AsyncMock(return_value=injected_response)
mod_inject = AuditModule({}, inner_inject)

stream = io.StringIO()
h = logging.StreamHandler(stream)
audit_logger.addHandler(h)
audit_logger.setLevel(logging.INFO)

await mod_inject.invoke([Message(role="user", content="hi")])

log_output = stream.getvalue()
audit_logger.removeHandler(h)
audit_logger.setLevel(logging.WARNING)

print(f"Log output: {repr(log_output.strip())}")
print(f"\nNewline escaped: {'\\n' in log_output}")
print("The injected log line is part of the model= field, not a separate line")

---
## 11. Content Logging Sanitization

Even when content logging is opted in, the output is sanitized.

In [ ]:
# Response with control characters in content
evil_response = LLMResponse(
    content="line1\nline2\rline3\tend",
    usage=Usage(input_tokens=10, output_tokens=5, total_tokens=15),
    model="test",
    stop_reason="end_turn",
)

inner_evil = MagicMock(spec=LLMProvider)
inner_evil.name = "test"
inner_evil.invoke = AsyncMock(return_value=evil_response)
mod_evil = AuditModule({"include_response": True}, inner_evil)

stream = io.StringIO()
h = logging.StreamHandler(stream)
h.setLevel(logging.DEBUG)
audit_logger.addHandler(h)
audit_logger.setLevel(logging.DEBUG)

await mod_evil.invoke([Message(role="user", content="hi")])

log_output = stream.getvalue()
audit_logger.removeHandler(h)
audit_logger.setLevel(logging.WARNING)

print("Control chars in opt-in content are escaped:")
for line in log_output.strip().split('\n'):
    if 'response' in line.lower():
        print(f"  {line.strip()}")

---
## 12. Exception Propagation

AuditModule never swallows exceptions — they propagate to the agent.

In [ ]:
inner_fail = MagicMock(spec=LLMProvider)
inner_fail.name = "test"
inner_fail.invoke = AsyncMock(side_effect=RuntimeError("provider failure"))
mod_fail = AuditModule({}, inner_fail)

try:
    await mod_fail.invoke([Message(role="user", content="hi")])
except RuntimeError as e:
    print(f"Exception propagated: {e}")
    print("Audit doesn't swallow errors — agent sees the failure")

---
## 13. Registry Integration

In [ ]:
from arcllm.registry import load_model, clear_cache

os.environ.setdefault("ANTHROPIC_API_KEY", "test-key")
os.environ.setdefault("OPENAI_API_KEY", "test-key")
clear_cache()

# Enable audit
model = load_model("anthropic", audit=True)
print(f"Type: {type(model).__name__}")
print(f"Inner: {type(model._inner).__name__}")

In [ ]:
clear_cache()

# Audit with content opt-in
model = load_model("anthropic", audit={"include_messages": True})
print(f"Type: {type(model).__name__}")
print(f"Include messages: {model._include_messages}")
print(f"Include response: {model._include_response}")

In [ ]:
clear_cache()

# Full 6-layer stack
model = load_model(
    "anthropic",
    telemetry=True,
    audit=True,
    retry=True,
    fallback=True,
    rate_limit=True,
)

# Walk the stack
layer = model
depth = 0
while hasattr(layer, '_inner'):
    indent = "  " * depth
    print(f"{indent}{type(layer).__name__}")
    layer = layer._inner
    depth += 1
print(f"{'  ' * depth}{type(layer).__name__}")

print(f"\n6 layers total")

Why Audit sits between Telemetry and Retry:

```
Telemetry (wall-clock + cost — outermost)
  └─ Audit (compliance metadata — sees retried/fallback'd result)
      └─ Retry (may retry multiple times)
          └─ Fallback (may switch provider)
              └─ RateLimit (may wait for token)
                  └─ Adapter (HTTP call)
```

Audit logs the **final** result the agent receives, not intermediate retry attempts.

---
## 14. Implementation Details

In [ ]:
import inspect

print("=== AuditModule.invoke ===")
print(inspect.getsource(AuditModule.invoke))

In [ ]:
print("=== AuditModule.__init__ ===")
print(inspect.getsource(AuditModule.__init__))

In [ ]:
print("=== log_structured() ===")
print(inspect.getsource(log_structured))

---
## 15. Live API Call with Audit

Real Anthropic API call with `audit=True`.

In [ ]:
from dotenv import load_dotenv
load_dotenv(os.path.join(os.getcwd(), '..', '.env'))

has_key = bool(os.environ.get("ANTHROPIC_API_KEY")) and os.environ.get("ANTHROPIC_API_KEY") != "test-key"
print(f"Anthropic API key: {'found' if has_key else 'not found (skip live tests)'}")

In [ ]:
if has_key:
    clear_cache()
    
    # Set up logging
    h = logging.StreamHandler()
    h.setFormatter(logging.Formatter("%(levelname)s %(name)s: %(message)s"))
    audit_logger = logging.getLogger("arcllm.modules.audit")
    audit_logger.addHandler(h)
    audit_logger.setLevel(logging.INFO)
    
    model = load_model("anthropic", audit=True)
    print(f"Stack: {type(model).__name__}({type(model._inner).__name__})")
    print()
    
    response = await model.invoke([
        Message(role="system", content="You are a helpful assistant."),
        Message(role="user", content="What is 2+2? Just the number."),
    ])
    
    print(f"\nResponse: {response.content}")
    print("\nNotice: audit log shows message_count=2 but NO message content")
    
    audit_logger.removeHandler(h)
    await model._inner.close()
else:
    print("Skipped — no API key")

In [ ]:
if has_key:
    clear_cache()
    
    # Audit + Telemetry together — both log independently
    h = logging.StreamHandler()
    h.setFormatter(logging.Formatter("%(levelname)s %(name)s: %(message)s"))
    
    for logger_name in ["arcllm.modules.audit", "arcllm.modules.telemetry"]:
        lgr = logging.getLogger(logger_name)
        lgr.addHandler(h)
        lgr.setLevel(logging.INFO)
    
    model = load_model("anthropic", audit=True, telemetry=True)
    
    # Walk the stack
    layers = []
    layer = model
    while hasattr(layer, '_inner'):
        layers.append(type(layer).__name__)
        layer = layer._inner
    layers.append(type(layer).__name__)
    print(f"Stack: {' → '.join(layers)}")
    print()
    
    response = await model.invoke([
        Message(role="user", content="Say 'audit works' in exactly those words.")
    ])
    
    print(f"\nResponse: {response.content}")
    print("\nTwo log lines above: one from Telemetry (timing/cost), one from Audit (metadata)")
    
    for logger_name in ["arcllm.modules.audit", "arcllm.modules.telemetry"]:
        lgr = logging.getLogger(logger_name)
        lgr.removeHandler(h)
    
    # Close innermost adapter
    innermost = model
    while hasattr(innermost, '_inner'):
        innermost = innermost._inner
    await innermost.close()
else:
    print("Skipped — no API key")

In [ ]:
if has_key:
    clear_cache()
    
    # Full stack with all modules
    h = logging.StreamHandler()
    h.setFormatter(logging.Formatter("%(levelname)s %(name)s: %(message)s"))
    for logger_name in ["arcllm.modules.audit", "arcllm.modules.telemetry"]:
        lgr = logging.getLogger(logger_name)
        lgr.addHandler(h)
        lgr.setLevel(logging.INFO)
    
    model = load_model(
        "anthropic",
        telemetry=True,
        audit=True,
        retry=True,
        rate_limit=True,
    )
    
    layers = []
    layer = model
    while hasattr(layer, '_inner'):
        layers.append(type(layer).__name__)
        layer = layer._inner
    layers.append(type(layer).__name__)
    print(f"Stack: {' → '.join(layers)}")
    print()
    
    response = await model.invoke([
        Message(role="user", content="What color is the sky? One word.")
    ])
    
    print(f"\nResponse: {response.content}")
    
    for logger_name in ["arcllm.modules.audit", "arcllm.modules.telemetry"]:
        lgr = logging.getLogger(logger_name)
        lgr.removeHandler(h)
    
    innermost = model
    while hasattr(innermost, '_inner'):
        innermost = innermost._inner
    await innermost.close()
else:
    print("Skipped — no API key")

---
## Summary

| Component | What | Why |
|-----------|------|-----|
| `AuditModule` | Logs audit metadata per `invoke()` | Federal compliance — every LLM interaction has a trail |
| PII-safe default | No raw content logged | NIST 800-53 AU-3: audit without new PII exposure |
| `include_messages` | Opt-in message content at DEBUG | Double opt-in for dev/staging environments |
| `include_response` | Opt-in response content at DEBUG | Same double opt-in safety |
| `_sanitize()` | Escapes `\n`, `\r`, `\t` | Prevents log injection attacks |
| `log_structured()` | Shared helper for all modules | DRY, consistent format, `None` omission, level gating |
| `validate_log_level()` | Shared validation | DRY between Telemetry and Audit |
| `_VALID_CONFIG_KEYS` | Strict key checking | Catches typos at construction time |
| Conditional fields | tools_provided, tool_calls | Only appear when relevant — reduces log noise |
| Stack position | Between Telemetry and Retry | Logs final result, not intermediate retry attempts |

**Config**:
```python
load_model("anthropic", audit=True)                                  # PII-safe metadata only
load_model("anthropic", audit={"include_messages": True})            # + message content at DEBUG
load_model("anthropic", audit={"include_response": True})            # + response content at DEBUG
load_model("anthropic", audit={"log_level": "DEBUG"})                # Custom level
load_model("anthropic", audit=False)                                 # Disable
```

**Log output** (default):
```
INFO  arcllm.modules.audit: Audit | provider=anthropic model=claude-sonnet-4-20250514
  message_count=2 stop_reason=end_turn tools_provided=1 tool_calls=2 content_length=142
```

**Test count**: 311 passed, 1 skipped (19 audit tests + 5 logging helper tests + 3 registry tests)